In [1]:
import gym
from gym import error, spaces, utils
from gym.utils import seeding
import os
import pybullet as p
import pybullet_data
import numpy as np
import matplotlib.pyplot as plt
import math
import random
import time

In [2]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Input
from keras.optimizers import Adam
from rl.agents.dqn import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

C:\Users\SERVER\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\SERVER\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\SERVER\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\SERVER\Anaconda3\lib\site-packages\tensorflow\python\framew

In [3]:
class BikeEnv(gym.Env):
    def __init__(self):
        p.connect(p.GUI)
        p.resetDebugVisualizerCamera(cameraDistance=10, cameraYaw=0, cameraPitch=-40, cameraTargetPosition=[0.55,-0.35,0.2])
        self.action_space = spaces.MultiDiscrete([ 3, 2, 2 ])
        self.observation_space = spaces.Box(np.array([-1000]*9), np.array([1000]*9))
        self.timestep = 1./240.
        
    def step(self, action):
        if action[0] == 0 :  
            p.setJointMotorControl2(self.pid, 0, p.VELOCITY_CONTROL, targetVelocity=0, force=0)
        if action[0] == 1 :  
            p.setJointMotorControl2(self.pid, 0, p.VELOCITY_CONTROL, targetVelocity=1, force=1)
        if action[0] == 2 :  
            p.setJointMotorControl2(self.pid, 0, p.VELOCITY_CONTROL, targetVelocity=-1, force=1)
            
        if action[1] == 0 :  
            p.setJointMotorControl2(self.pid, 1, p.VELOCITY_CONTROL, targetVelocity=0, force=0)  
        if action[1] == 1 :  
            p.setJointMotorControl2(self.pid, 1, p.VELOCITY_CONTROL, targetVelocity=5, force=5)
         
        if action[2] == 0 :  
            p.setJointMotorControl2(self.pid, 2, p.VELOCITY_CONTROL, targetVelocity=0, force=0) 
        if action[2] == 1 :  
            p.setJointMotorControl2(self.pid, 2, p.VELOCITY_CONTROL, targetVelocity=5, force=5)    
    
        p.stepSimulation()
        time.sleep(self.timestep)
        
        state = p.getLinkState(self.pid,0)[0]
        if state[2] <= 0.7 or  state[2] >= 2:
            reward = -100
            done = True
        else :
            reward = math.sqrt((self.origin[0]-state[0])**2+(self.origin[1]-state[1])**2)
            done = False
        self.origin = state 
        
        observation = p.getLinkState(self.pid,0)[0]+p.getLinkState(self.pid,1)[0]+p.getLinkState(self.pid,2)[0]
        state_object, _ = p.getBasePositionAndOrientation(self.pid)
        info = {'x':state_object[0],'y':state_object[1],'z':state_object[2]}
        return observation, reward, done, info
            
        
    def reset(self):
        p.resetSimulation()

        urdfRootPath = pybullet_data.getDataPath()
        planeUid = p.loadURDF(os.path.join(urdfRootPath,"plane.urdf"), basePosition=[0,0,0])
        self.pid = p.loadURDF(os.path.join(urdfRootPath, "bicycle/bike.urdf"),basePosition=[0,0,1])
        self.origin = p.getLinkState(self.pid,0)[0]
        p.setGravity(0,0,-10)
        p.setRealTimeSimulation(0)

        observation =p.getLinkState(self.pid,0)[0]+p.getLinkState(self.pid,1)[0]+p.getLinkState(self.pid,2)[0]
        
        p.configureDebugVisualizer(p.COV_ENABLE_RENDERING,1)
        return observation
        
    def render(self, mode='human'):
        view_matrix = p.computeViewMatrixFromYawPitchRoll(cameraTargetPosition=[0.7,0,0.05],
                                                            distance=.7,
                                                            yaw=90,
                                                            pitch=-70,
                                                            roll=0,
                                                            upAxisIndex=2)
        proj_matrix = p.computeProjectionMatrixFOV(fov=60,
                                                     aspect=float(960) /720,
                                                     nearVal=0.1,
                                                     farVal=100.0)
        (_, _, px, _, _) = p.getCameraImage(width=960,
                                              height=720,
                                              viewMatrix=view_matrix,
                                              projectionMatrix=proj_matrix,
                                              renderer=p.ER_BULLET_HARDWARE_OPENGL)

        rgb_array = np.array(px, dtype=np.uint8)
        rgb_array = np.reshape(rgb_array, (720,960, 4))

        rgb_array = rgb_array[:, :, :3]
        return rgb_array

    def close(self):
        p.disconnect()

In [4]:
env = BikeEnv()
env.reset()
for _ in range(1000):
    env.render()
    observation, reward, done, info = env.step(env.action_space.sample()) # take a random action
    if done :
        env.reset()
env.close()

C:\Users\SERVER\Anaconda3\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
